# Mathematical Optimization with `JuMP`

## Example Problem: Economic Dispatch in Power Systems

Imagine your task is to operate an electric power grid. You need to supply a certain amount of electrical power to the grid. You have in front of you:
- a set of generators
  - each with different costs and maximum capacities

How much power should you get from which generator to satisfy the power demand while at the same time saving cost? This is the Economic Dispatch Problem.


The Economic Dispatch problem becomes more complex with the increasing size of the power system, uncertainty of renewable energy sources, and the introduction of smart grids and electric vehicles. As such, it remains an active area of research in power systems engineering. 

For the following example, we will solve a simple version of the problem using the `JuMP` package.






## Simplified Economic Dispatch Problem


A simple version of he Economic Dispatch Problem (EDP) can be mathematically formulated as follows:

**Objective Function:**

Minimize: 

$$
C(P_1, P_2, ..., P_k) = \sum_{i=1}^{k} a_iP_i^2
$$

Where:

- $C(P_1, P_2, ..., P_k)$ is the total cost function, which is the sum of the cost functions of all "k" generators.
- $a_iP_i^2$ is the simplified cost function of the $i$-th generator. 
- $P_i$ is the power output of the $i$-th generator.

**Subject to:**

1. _Power Balance Constraint_:

$$
\sum_{i=1}^{k} P_i = P_D
$$

Where $P_D$ is the total power demand that needs to be satisfied.

2. _Generator Capacity Constraints_:

For each Generator $i$: $P_{min_i} \leq P_i \leq P_{max_i}$

These constraints ensure the power generated by each generator is within its capacity.

## Vector/Matrix Form 

The same problem can be expressed more concisely in vector/matrix form.

**Objective Function:**

Minimize: 

$$
C(P) = \frac{1}{2} P^TAP
$$

Where:

- $P$ is a $k \times 1$ vector representing the power outputs of all generators: $P = [P_1, P_2, ..., P_k]^T$.
- $A$ is a $k \times k$ diagonal matrix where the $i$-th diagonal element is the quadratic cost coefficient $a_i$ of the $i$-th generator.

**Subject to:**

1. Power Balance Constraint:

$$
1^TP = P_D
$$

Where $1$ is a vector of ones.

2. Generator Capacity Constraints:

$$
P_{min} \leq P \leq P_{max}
$$

Where $P_{min}$ and $P_{max}$ are $k \times 1$ vectors representing the minimum and maximum capacity of each generator respectively.

## Example Instance

In [1]:
# Number of generators
k = 3

# Quadratic cost coefficients for each generator
A = [0.5, 0.3, 0.4]

# Minimum and maximum capacity for each generator
P_min = [30, 20, 40]
P_max = [100, 80, 120]

# Total power demand
P_D = 150

150

## Solving the Problem with JuMP

We'll be using the JuMP package in Julia, which provides a high-level interface for mathematical programming, and the Ipopt solver, which is capable of solving nonlinear optimization problems.

1. Install and import the necessary packages


In [2]:
import Pkg
Pkg.add("JuMP")
Pkg.add("Ipopt")

using JuMP, Ipopt

┌ Warning: could not download https://pkg.julialang.org/registries
│   exception = Downloads.RequestError("https://pkg.julialang.org/registries", 6, "Could not resolve host: pkg.julialang.org", Downloads.Response(nothing, "https://pkg.julialang.org/registries", 0, "", Pair{String, String}[]))
└ @ Pkg.Registry /opt/homebrew/Cellar/julia/1.9.4/share/julia/stdlib/v1.9/Pkg/src/Registry/Registry.jl:69
   Resolving package versions...
  No Changes

 to `~/Documents/Work/Training/point8/data-science-learning-paths/Project.toml`
  No Changes to `~/Documents/Work/Training/point8/data-science-learning-paths/Manifest.toml`


   Resolving package versions...


┌ Warning: could not download https://pkg.julialang.org/registries
│   exception = Downloads.RequestError("https://pkg.julialang.org/registries", 6, "Could not resolve host: pkg.julialang.org", Downloads.Response(nothing, "https://pkg.julialang.org/registries", 0, "", Pair{String, String}[]))
└ @ Pkg.Registry /opt/homebrew/Cellar/julia/1.9.4/share/julia/stdlib/v1.9/Pkg/src/Registry/Registry.jl:69


     Cloning [b6b21f68-93f8-5de0-b562-5493be1d77c9] Ipopt from https://github.com/jump-dev/Ipopt.jl.git


Pkg.Types.PkgError: failed to clone from https://github.com/jump-dev/Ipopt.jl.git, error: GitError(Code:ERROR, Class:Net, failed to resolve address for github.com: nodename nor servname provided, or not known)

2. **Define the model**: Here, we define model to be a new optimization problem. We specify that we want to use the Ipopt solver.

In [3]:
model = Model(Ipopt.Optimizer)

UndefVarError: UndefVarError: `Ipopt` not defined

3. **Define the decision variables**: The decision variables are the power outputs of the generators, represented by the array P. The power output of each generator should be greater than or equal to 0.

In [4]:
@variable(model, P[1:k] >= 0)

LoadError: LoadError: UndefVarError: `@variable` not defined
in expression starting at /Users/cls/Documents/Work/Training/point8/data-science-learning-paths/notebooks/julia/julia-optimization.ipynb:1

1. **Set the objective function**:  We set the objective of our problem, which is to minimize the total cost of power generation. We use the @NLobjective macro because our objective function is nonlinear. The total cost is the sum of the cost of each generator, which is a quadratic function of the power output of the generator.

In [5]:
@NLobjective(model, Min, sum(A[i]*P[i]^2 for i in 1:k))

LoadError: LoadError: UndefVarError: `@NLobjective` not defined
in expression starting at /Users/cls/Documents/Work/Training/point8/data-science-learning-paths/notebooks/julia/julia-optimization.ipynb:1

5. **Set the constraints**: We add constraints to our problem. The first constraint is the _power balance constraint_, which states that the total power generated must equal the total power demand. The second set of constraints are the _generator capacity constraints_. These state that the power output of each generator must be within its minimum and maximum capacity.

In [6]:
# Power balance constraint
@constraint(model, sum(P[i] for i in 1:k) == P_D)

# Generator capacity constraints
for i in 1:k
    @constraint(model, P[i] >= P_min[i])
    @constraint(model, P[i] <= P_max[i])
end

LoadError: LoadError: UndefVarError: `@constraint` not defined
in expression starting at /Users/cls/Documents/Work/Training/point8/data-science-learning-paths/notebooks/julia/julia-optimization.ipynb:2

1. **Solve the problem**:  To solve the problem we use the `optimize!`` function in JuMP. This function takes the model we've defined, including the objective function and constraints, and solves it using the specified optimizer, which is _Ipopt_ in our case.

In [7]:
optimize!(model)

UndefVarError: UndefVarError: `optimize!` not defined

Once the solver has found an optimal solution, you can retrieve the optimal power outputs with the value function:

In [8]:
value.(P)

UndefVarError: UndefVarError: `value` not defined

---
_This notebook is licensed under a [Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International (CC BY-NC-SA 4.0)](https://creativecommons.org/licenses/by-nc-sa/4.0/). Copyright © 2018-2024 [Point 8 GmbH](https://point-8.de)_